## 使用一个神经网路解决 FizzBuzz 问题

输入一个数字：
 - 可以被 15 整除就回传 FizzBuzz；
 - 可以被 3 整除就回传 Fizz；
 - 被 5 整除就回传 Buzz；
 - 都不能整除就回传原本的数字

## 用程式实做的话会长这样

In [1]:
def fizz_buzz(num):
    if num % 15 == 0:
        return 'FizzBuzz'
    elif num % 3 == 0:
        return 'Fizz'
    elif num % 5 == 0:
        return 'Buzz'
    else:
        return str(num)
    
print(fizz_buzz(45))
print(fizz_buzz(48))
print(fizz_buzz(50))

FizzBuzz
Fizz
Buzz


## 其实就是一个简单的分类问题

In [2]:
def fizz_buzz(num):
    if num % 15 == 0:
        return 0 # 'FizzBuzz'
    elif num % 3 == 0:
        return 1 # 'Fizz'
    elif num % 5 == 0:
        return 2 # 'Buzz'
    else:
        return 3 # ''

## 来看看深度学习怎么解决这皮毛问题

In [3]:
import random
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

BATCH = 32
DIGITS = 10
EPOCH = 100
DATASET_SIZE = 3000
CLASSES = ['FizzBuzz', 'Fizz', 'Buzz', '']

## 深度學習很難嗎？其實網路結構就這幾行

In [4]:
class FizzBuzz(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(FizzBuzz, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_channel, 1024), ## 全连接层
            nn.Dropout(0.5),             ## Dropout 避免 overfitting
            nn.ReLU(),                   ## Activation 函式
            nn.Linear(1024, 1024),       ## 全连接层
            nn.Dropout(0.5),             ## Dropout 避免 overfitting
            nn.ReLU(),                   ## Activation 函式
            nn.Linear(1024, out_channel) ## 全连接层，以此例来说 out_channel 应该为 4
        )

    def forward(self, x):
        x = self.layers(x)
        return x

## 可参考 lost 函式清单
 - https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [5]:
def training(model, optimizer, training_data):
    model.train()

    for data, label in training_data:
        data = Variable(torch.FloatTensor(data))
        label = Variable(torch.LongTensor(label))

        out = model(data)
        optimizer.zero_grad()
        ## 常用分类问题的 loss 函式
        classification_loss = F.cross_entropy(out, label) 
        classification_loss.backward()
        optimizer.step()

In [7]:
m = FizzBuzz(DIGITS, 4)
print(m) 

FizzBuzz(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=1024, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=4, bias=True)
  )
)


## 检查一下准确率

In [8]:
def testing(model, data):
    model.eval()
    loss = []
    correct = 0

    for x, y in data:
        x = Variable(torch.FloatTensor(x))
        y = Variable(torch.LongTensor(y))

        out = model(x)
        loss += [F.cross_entropy(out, y).data.item()]
        pred = out.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).sum()

    avg_loss = sum(loss) / len(loss)

    return {
        'accuracy': 100.0 * correct/(len(loss)*BATCH),
        'avg_loss': avg_loss
    }

### 输入格式转换成二进制

In [9]:
def encoder(num):
    return list(map(lambda x: int(x), ('{:0' + str(DIGITS) + 'b}').format(num))) 

print(encoder(2088))

[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]


## 制造一大堆训练资料

In [10]:
def make_data(num_of_data, batch_size):
    xs = []
    ys = []
    for _ in range(num_of_data):
        x = random.randint(0, 2**DIGITS-1)
        ## 输入资料
        xs += [encoder(x)]
        ## 预测目标
        ys += [fizz_buzz(x)]

    data = []
    for b in range(num_of_data//batch_size):
        xxs = xs[b*batch_size:(b+1)*batch_size]
        yys = ys[b*batch_size:(b+1)*batch_size]
        data += [(xxs, yys)]
        
    return data

In [11]:
training_data = make_data(DATASET_SIZE, BATCH)
testing_data = make_data(100, BATCH)

In [12]:
optimizer = optim.SGD(m.parameters(), lr=0.02, momentum=0.9)

print('==== 开始训练 ====')
for epoch in range(1, EPOCH + 1):
    training(m, optimizer, training_data)
    res = testing(m, testing_data)
    print('Epoch {}/{}, Loss: {:.5f}, Accuracy: {:.2f}%'.format(
            epoch,
            EPOCH,
            res['avg_loss'],
            res['accuracy'],
        ), end='\r')


==== 开始训练 ====


## 即时测试

In [13]:
def interactive_test(model):
    while True:
        num = input()
        if num == 'q':
            print('Bye~')
            return
        if int(num) >= 2**DIGITS:
            print('请输入一个小于 {} 的数字'.format(2**DIGITS))
            continue

        ans = fizz_buzz(int(num))
        x = Variable(torch.FloatTensor([encoder(int(num))]))

        predict = model(x).data.max(1, keepdim=True)[1]
        print('预测为: {}, 答案为: {}'.format(CLASSES[predict[0][0]], CLASSES[ans]))

In [14]:
print('[提示] 请输入一个小于 {} 的数字. (或输入 "q" 离开)'.format(2**DIGITS))
interactive_test(m)

[提示] 请输入一个小于 1024 的数字. (或输入 "q" 离开)
258
预测为: Fizz, 答案为: Fizz
333
预测为: Fizz, 答案为: Fizz
150
预测为: FizzBuzz, 答案为: FizzBuzz
50
预测为: Buzz, 答案为: Buzz
q
Bye~
